# Generating SVGs from Niworkflows

This notebook is for exploring the nipype/niworkflows visualization libraries to generate the required image for QC'ing HCP pipelines.

From looking at fmriprep/niworkflows/nipype it looks like the predominant method for generating visualizations is to create sub-classes that base a set of **ReportCapable** classes derived from `niworkflows.interfaces.report_base`. 

Custom classes can also be created in the style of the classes found in `niworkflows.interfaces.report_base`, they just need to subclass `nipype.interfaces.mixins.reporting.ReportCapableInterface`.

Since our results are pre-generated, we just need a node object class which supports a clean identity mapping (i.e kinda like `nipype.util.IdentityInterface`) but generates visualizations of our desired type as a side-effect.

In [1]:
# Import requisite classes
import niworkflows.interfaces.report_base as nrc
from nipype.interfaces.utility import IdentityInterface
from nipype.interfaces.base import traits
from nipype.interfaces.mixins import reporting
from nipype.interfaces.base import DynamicTraitedSpec
from nipype.interfaces.base import File
from nipype.interfaces.base import BaseInterfaceInputSpec

In [2]:
import pdb

### The IRegInputSpecRPT input specification

Adding the `niworkflow.interfaces.report_base._SVGReportCapableInputSpec` mixin class extends our basic identity input spec with the following specs:

- `out_report` which of type `trait.File(name, usedefault, desc)`
- `compress_report` which is of type `trait.Enum(["auto",True,False], usedefault, desc)`

`traits` are just ways of _incorporating type-like specifications into `nipype`_

Not only do we want to implement properties that allow us to spit out reports, we also want to be able to add image inputs. To deal with this we'll add additional `bg_nii` and `fg_nii` properties. Both of these are `File` traits that:

1. Must exist `exists=True`
2. Have no defaults `defaults=False`
3. Absolute paths will be resolved to relative paths `resolve=True`
4. Have descriptions specified by `desc`
5. Are mandatory inputs `mandatory=True`

In [3]:
class _IRegInputSpecRPT(nrc._SVGReportCapableInputSpec,
                       BaseInterfaceInputSpec):
    
    '''
    Input specification for IRegRPT, implements:
    
    bg_nii: Input background NIFTI image
    fg_nii: Input foreground NIFTI image
    contour: Optional outlining NIFTI Image
    
    Bases _SVGReportCapableInputSpec which implements:
    
    out_report: Filename trait
    compress_report: ["auto", true, false]
    
    '''
    bg_nii = File(exists=True,
                  usedefault=False,
                  resolve=True,
                  desc='Background image of SVG',
                  mandatory=True)
    
    fg_nii = File(exists=True,
                  usedefault=False,
                  resolve=True,
                  desc='Foreground image of SVG',
                  mandatory=True)
    
    contour = File(exists=True,
                  udedefault=False,
                  resolve=True,
                  desc='Contours to include in image',
                  mandatory=False)

### The IRegOutputSpecRPT output specification

Next we want to specify the output specification for our `RPT` class. This is done by just create an interface object that mixes in the `nipype.interfaces.mixins.reporting.ReportCapableOutputSpec` object which adds the following property:

- `out_report` of type `File(name, usedefault, hash_files, desc)`, the "report" actually just refers to the `.svg` file that will be generated.

Since we're not _actually outputting anything here, just relying on a side-effect,_ we don't have to add additional output traits

In [4]:
class _IRegOutputSpecRPT(reporting.ReportCapableOutputSpec):
    pass

Ultimately using both the mixin classes allows us to interface with niworkflows and nipype's visualization toolkit in a consistent, standard manner.

***

### Creating a nipype concrete class

Now that we have an IO specification that implement a standard way of inputs and outputs the next step is to create a class that can interface with Nipype's engine. We define a class `IRegRPT` which:

1. Mixes in `niworkflows.interfaces.report_base.RegistrationRC` which provides a `_post_hook` method to auto-generate Registration (SVG flicker animated) images as a side-effect


2. Literally does nothing but set variables that the `RegistrationRC` will use during its `_post_run_hook` routine... after-all the images are already aligned and no additional operations are needed. 


---
One thing to note is that we have the flexibility to perform additional transformations within this class, this is what `niworkflows` typically does. 

In addition you can base off of other established `nipype.interface` classes to extend their functionality to add report capability. This is what is done by classes like `niworkflows.interfaces.register.BBRegisterRPT` 

***


Finally we must implement the `_run_interface(self,runtime) --> runtime` method which is called by the `BaseInterface` class' `run()` method. Since we don't want to perform any actual computation, we make this function do literally nothing.

In [21]:
class IRegRPT(nrc.RegistrationRC):
    '''
    Class to generate registration images from pre-existing
    NIFTI files. 
    
    Effectively acts as an Identity node with report
    generation as a side-effect.
    '''

    # Use our declared IO specs
    input_spec = _IRegInputSpecRPT
    output_spec = _IRegOutputSpecRPT
    
    
    
    def _post_run_hook(self, runtime):
        
        '''
        Do nothing but propogate properties
        to (first) parent class of IRegRPT
        that is nrc.RegistrationRC
        '''
        
        # Set variables for `nrc.RegistrationRC`
        self._fixed_image = self.inputs.bg_nii
        self._moving_image = self.inputs.fg_nii
        
        # Propogate to RegistrationRC superclass
        return super(IRegRPT, self)._post_run_hook(runtime)
    


    def _run_interface(self, runtime):
        return runtime

In [22]:
# Set up inputs
pre='../data/sub-CMHP001/ses-01/anat/sub-CMHP001_ses-01_desc-preproc_T1w.nii.gz'
post='../data/sub-CMHP001/ses-01/anat/sub-CMHP001_ses-01_desc-aseg_dseg.nii.gz'

In [23]:
# Set up input spec and generate report as side-effect
regrpt = IRegRPT(generate_report=True)
regrpt.inputs.bg_nii = pre
regrpt.inputs.fg_nii = post
regrpt.inputs.out_report = '../tmp/test.svg'

In [ ]:
# Run no actual computation, but generate report as side-effect
regrpt.run()

# This should work successfully!